In [21]:
from pydriller import RepositoryMining

In [22]:
export_config = {
    "url": "../data/react-vis",
    "collection": 'react-vis'
}

In [23]:
mongo_config = {
    "server_address": "localhost",
    "server_port": 27017,
    "db": "git-vis"
}

In [24]:
from pydriller.domain.commit import ModificationType
from pymongo import MongoClient

def export2mongo(export_config, mongo_config):
    
    # open mongo db connection
    client = MongoClient(mongo_config["server_address"], mongo_config["server_port"])
    db = client['db']
    
    # drop collection if it already exists
    collist = db.list_collection_names()
    if export_config['collection'] in collist:
        db[export_config["collection"]].drop()
        
    # create collection
    tbl = db[export_config['collection']]

    # iterate through all the commits and store in mongo
    for commit in RepositoryMining(path_to_repo=[export_config['url']]).traverse_commits():
        # store commit info
        commit_info = {
            "commit_hash": commit.hash,
            "author_email": commit.author.email,
            "author_date": commit.author_date,
            "commit_msg": commit.msg
        }
        # store modification on each file
        for modification in commit.modifications:
            doc = {**commit_info, **{
                "new_path": modification.new_path,
                "change_type": str(modification.change_type),
                "diff": modification.diff,
                "lines_added": modification.added,
                "lines_removed": modification.removed 
            }}
            tbl.insert_one(doc)

In [25]:
export2mongo(export_config, mongo_config)